##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 检查 TensorFlow 图

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tensorboard/graphs"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 TensorFlow.google.cn 访问</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tensorboard/graphs.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tensorboard/graphs.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 GitHub 查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tensorboard/graphs.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载此 notebook</a>
  </td>
</table>

## 概述

TensorBoard 的 **图仪表盘** 是检查 TensorFlow 模型的强大工具。您可以快速查看模型结构的预览图，并确保其符合您的预期想法。 您还可以查看操作级图以了解 TensorFlow 如何理解您的程序。检查操作级图可以使您深入了解如何更改模型。例如，如果训练进度比预期的慢，则可以重新设计模型。

本教程简要概述了如何在 TensorBoard 的 图仪表板中生成图诊断数据并将其可视化。您将为 Fashion-MNIST 数据集定义和训练一个简单的 Keras 序列模型，并学习如何记录和检查模型图。您还将使用跟踪API为使用新的 `tf.function` 注释创建的函数生成图数据。

## 设置

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0-dev20190304


In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## 定义一个 Keras 模型

在此示例中，分类器是一个简单的四层顺序模型。

In [ ]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

下载并准备训练数据

In [ ]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0

## 训练模型并记录数据

训练之前，请定义 [Keras TensorBoard callback](https://tensorflow.google.cn/api_docs/python/tf/keras/callbacks/TensorBoard), 并指定日志目录。通过将此回调传递给 Model.fit(), 可以确保在 TensorBoard 中记录图形数据以进行可视化。

In [26]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.7073 - accuracy: 0.7577
Epoch 2/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4838 - accuracy: 0.8297
Epoch 3/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4439 - accuracy: 0.8417
Epoch 4/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4230 - accuracy: 0.8478
Epoch 5/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4124 - accuracy: 0.8520


## op-level graph

启动 TensorBoard，然后等待几秒钟以加载 UI。通过点击顶部的 “graph” 来选择图形仪表板。 

In [ ]:
%tensorboard --logdir logs

默认情况下，TensorBoard 显示 **op-level图**。（在左侧，您可以看到已选择 “Default” 标签。）请注意，图是倒置的。 数据从下到上流动，因此与代码相比是上下颠倒的。 但是，您可以看到该图与 Keras 模型定义紧密匹配，并具有其他计算节点的额外边缘。

图通常很大，因此您可以操纵图的可视化效果：

* 滚动到 **zoom** 来放大和缩小
* 拖到 **pan** 平移
* 双击切换 **node expansion** 来进行节点扩展（一个节点可以是其他节点的容器）

您还可以通过单击节点来查看元数据。这使您可以查看输入，输出，形状和其他详细信息。


<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation.png?raw=1"/>

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation_detail.png?raw=1"/>

## 概念图

除了执行图，TensorBoard 还显示一个“概念图”。 这只是 Keras 模型的视图。 如果您要重新使用保存的模型并且想要检查或验证其结构，这可能会很有用。

要查看概念图，请选择 “keras” 标签。 在此示例中，您将看到一个折叠的 **Sequential** 节点。 双击节点以查看模型的结构：

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_tag_selection.png?raw=1"/> <br/>
<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_conceptual.png?raw=1"/>

## tf.functions 的图

到目前为止的示例已经描述了 Keras 模型的图，其中这些图是通过定义 Keras 层并调用 Model.fit() 创建的。

您可能会遇到需要使用 `tf.function` 注释来[[autograph]](https://tensorflow.google.cn/guide/function)的情况，即将 Python 计算函数转换为高性能 TensorFlow 图。对于这些情况，您可以使用 TensorBoard 中的 **TensorFlow Summary Trace API** 记录签名函数以进行可视化。

要使用 Summary Trace API ，请执行以下操作:

*   使用 `tf.function` 定义和注释功能
*   在函数调用站点之前立即使用 `tf.summary.trace_on()`
*   通过传递 `profiler=True` 将配置文件信息（内存，CPU时间）添加到图中
*   使用摘要文件编写器，调用 `tf.summary.trace_export()` 保存日志数据

然后，您可以使用 TensorBoard 查看函数的行为。

In [ ]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_autograph.png?raw=1"/>

现在，您可以看到 TensorBoard 所理解的函数结构。 单击 “Profile” 按钮以查看 CPU 和内存统计信息。